In [ ]:
!pip install -U bitsandbytes

In [4]:
!pip install transformers datasets peft trl bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 19.6 MB/s eta 0:00:00


In [1]:
!pip install huggingface_hub

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from datasets import load_dataset

# 使用datasets库直接加载我们上传的CSV文件
data_files = {"train": "/content/drive/MyDrive/Colab Notebooks/Chinese_Poetry.csv"}
raw_dataset = load_dataset("csv", data_files=data_files)

# 只需要诗歌的文本内容，也就是'Poem_Text'这一列
# 对数据集进行一次处理，变成SFTTrainer需要的格式
def process_function(examples):
    # 将'Poem_Text'列的内容格式化，SFTTrainer需要一个名为'text'的字段
    return {"text": [text for text in examples["Poem_Text"]]}

# 使用.map()方法应用这个处理函数
processed_dataset = raw_dataset.map(process_function, batched=True, remove_columns=raw_dataset['train'].column_names)
dataset = processed_dataset["train"]
print("从Kaggle加载并处理好的数据示例：")
print(dataset[0])
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
model_id = "Qwen/Qwen1.5-1.8B-Chat"

# 设置量化配置，用于节省显存
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto" # 自动把模型加载到GPU
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# 设置一个 padding token，用于填充数据
tokenizer.pad_token = tokenizer.eos_token

from peft import LoraConfig

# 配置LoRA参数
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # 针对模型中的特定层
    task_type="CAUSAL_LM",
)
from transformers import TrainingArguments
from trl import SFTTrainer

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./qwen-poet",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=500,
    logging_steps=10,
    report_to="none",
)

# 创建监督微调训练器
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_args,
)

trainer.train()
print("微调训练完成！")

# 准备一个测试提示
prompt_text = "月光下的长安城，"

# 使用分词器准备输入
inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda") # 确保输入在GPU上

# 生成文本
outputs = model.generate(**inputs, max_new_tokens=50)

# 解码并打印结果
print("微调后的模型回答：")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

从Kaggle加载并处理好的数据示例：
{'text': '此诗第1句，风月无边，情思缠绵。'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.954100
20,0.723500
30,0.450600
40,0.467000
50,0.438400
60,0.447200
70,0.452800
80,0.449500
90,0.437800
100,0.439900


微调训练完成！
微调后的模型回答：
月光下的长安城，繁华依旧。月光照耀在大雁塔上，洒下一片银光。大雁塔是唐代建筑的瑰宝，高约130米，四壁镶嵌着精美的浮雕和壁画，描绘了唐朝的历史


In [9]:
# =======================================================
#               A/B 对比测试代码
# =======================================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# 加载微调后的模型
finetuned_model = trainer.model


#重新加载一个未经微调的模型
base_model_id = "Qwen/Qwen1.5-1.8B-Chat"
quantization_config = BitsAndBytesConfig(load_in_4bit=True) # 使用和之前一样的量化配置

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

#用同一个问题，分别问两个模型
prompt_text = "床前明月光，"

# 准备输入
inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

# A组（微调后模型）作答
print("--- 正在使用【微调后】的模型生成... ---")
finetuned_outputs = finetuned_model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(finetuned_outputs[0], skip_special_tokens=True))

print("\n" + "="*50 + "\n")

# B组（原始模型）作答
print("--- 正在使用【原始】的模型生成... ---")
base_model_outputs = base_model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(base_model_outputs[0], skip_special_tokens=True))

--- 正在使用【微调后】的模型生成... ---
床前明月光，疑是地上霜。举头望明月，低头思故乡。这首诗的第一句是（）。 A. 嘉陵江畔 B. 珠穆朗玛峰 C. 长城 D. 拱


--- 正在使用【原始】的模型生成... ---
床前明月光，疑是地上霜。举头望明月，低头思故乡。

《静夜思》（唐）李白

床前明月光，疑是地上霜。举头望明月，低头思故乡。

——李白

床


In [4]:
output_directory = "./my_finetuned_poet_model"

# 使用之前创建的trainer对象来保存模型
trainer.save_model(output_directory)

print(f"模型已成功保存到 Colab 的 '{output_directory}' 文件夹中！")

模型已成功保存到 Colab 的 './my_finetuned_poet_model' 文件夹中！


In [5]:
import shutil
folder_to_zip = 'my_finetuned_poet_model'
zip_filename = 'my_finetuned_poet_model.zip'
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', folder_to_zip)

print(f"文件夹 '{folder_to_zip}' 已成功压缩为 '{zip_filename}'！")

文件夹 'my_finetuned_poet_model' 已成功压缩为 'my_finetuned_poet_model.zip'！


In [2]:
from huggingface_hub import login

login()

In [10]:
hub_model_id = "wushir828/Qwen1.5-1.8B-Chinese-Poet-LoRA"

finetuned_model = trainer.model
tokenizer = tokenizer

print(f"正在上传LoRA适配器到 {hub_model_id}...")
finetuned_model.push_to_hub(hub_model_id)

print(f"\n正在上传分词器到 {hub_model_id}...")
tokenizer.push_to_hub(hub_model_id)

print("\n上传完成！")

正在上传LoRA适配器到 wushir828/Qwen1.5-1.8B-Chinese-Poet-LoRA...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   4%|4         |  565kB / 12.6MB            


正在上传分词器到 wushir828/Qwen1.5-1.8B-Chinese-Poet-LoRA...


README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpsfek1ptc/tokenizer.json:  28%|##7       | 3.15MB / 11.4MB            


上传完成！
